<a href="https://colab.research.google.com/github/Shubhgupta000/Extracting_Experience_From_JDs/blob/main/Extracting_Experience_From_JDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas openpyxl spacy transformers sentence-transformers
!python -m spacy download en_core_web_sm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [1]:
import pandas as pd
from google.colab import files

# Upload the Excel file
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

# Read the Excel file
df = pd.read_excel(file_name, sheet_name='Sheet1')

# Display first few rows to verify
print(df.head())

Saving Job Descriptions.xlsx to Job Descriptions.xlsx
                                    Job Descriptions
0  Greetings from HDFC Sales (Subsidiary of HDFC ...
1  Dear Aspirants,\n\nOrcapod is looking for Fema...
2  â€¢ Involved with full life cycle of the recru...
3  **Job description**\n\n  \n\n\n  * Data Entry ...
4  **You're ideal for this role, if**\n\n  * You'...


In [19]:
print(df.columns)

Index(['Job Descriptions', 'processed_text'], dtype='object')


In [3]:
import re
import spacy

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    if not isinstance(text, str):
        return ""

    # Basic cleaning
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)  # Remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra whitespace

    # Lemmatization with spaCy
    doc = nlp(text)
    lemmatized = " ".join([token.lemma_ for token in doc])

    return lemmatized

# Apply preprocessing
df['processed_text'] = df['Job Descriptions'].apply(preprocess_text)

In [5]:
def extract_experience_spacy(text):
    doc = nlp(text)

    experience_phrases = []
    for sent in doc.sents:
        for token in sent:
            if token.text in ['experience', 'exp']:
                # Look for number patterns nearby
                for child in token.head.children:
                    if child.like_num:
                        experience_phrases.append(f"{child.text} {token.text}")

    return ", ".join(experience_phrases) if experience_phrases else None

In [6]:
from transformers import pipeline

qa_pipeline = pipeline("question-answering", model="deepset/bert-base-cased-squad2")

def extract_experience_bert(text):
    question = "What is the required experience for this job?"
    result = qa_pipeline(question=question, context=text)
    return result['answer'] if result['score'] > 0.5 else None

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [7]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def extract_experience_zero_shot(text):
    candidate_labels = ["0-2 years", "2-5 years", "5-8 years", "8-12 years", "12+ years"]
    result = classifier(text, candidate_labels)
    return result['labels'][0]

config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [9]:
def extract_experience_custom(text):
    doc = nlp(text)

    experience_ranges = []

    for token in doc:
        if token.text in ['experience', 'exp']:
            # Look for number patterns in the neighborhood
            for i in range(max(0, token.i-5), min(len(doc), token.i+5)):
                if doc[i].like_num:
                    # Check if it's part of a range
                    if i+1 < len(doc) and doc[i+1].text in ['-', 'to'] and i+2 < len(doc) and doc[i+2].like_num:
                        experience_ranges.append(f"{doc[i].text}-{doc[i+2].text} years")
                    else:
                        experience_ranges.append(f"{doc[i].text} years")

    return ", ".join(experience_ranges) if experience_ranges else None

In [10]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Define experience pattern sentences
experience_patterns = [
    "minimum 5 years of experience",
    "3-5 years experience required",
    "8+ years of relevant experience",
    "1-2 years experience needed",
    "10-12 years of work experience"
]

def extract_experience_sentence_transformer(text):
    # Encode text and patterns
    text_embedding = model.encode(text, convert_to_tensor=True)
    pattern_embeddings = model.encode(experience_patterns, convert_to_tensor=True)

    # Calculate similarities
    similarities = util.pytorch_cos_sim(text_embedding, pattern_embeddings)[0]

    # Get most similar pattern
    most_similar_idx = np.argmax(similarities)
    if similarities[most_similar_idx] > 0.5:
        return experience_patterns[most_similar_idx]
    return None

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Apply all models to the data
df['spacy_result'] = df['processed_text'].apply(extract_experience_spacy)
df['bert_result'] = df['Job Descriptions'].apply(extract_experience_bert)
df['zero_shot_result'] = df['processed_text'].apply(extract_experience_zero_shot)
df['custom_result'] = df['processed_text'].apply(extract_experience_custom)
df['sentence_transformer_result'] = df['processed_text'].apply(extract_experience_sentence_transformer)

# Display results
print(df[['processed_text', 'spacy_result', 'bert_result', 'zero_shot_result', 'custom_result', 'sentence_transformer_result']].head())

In [25]:
# Save to Excel
output_file = "experience_extraction_results.xlsx"
df.to_excel(output_file, index=False)

# Download the file
files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>